In [ ]:
import numpy as np
import keras
from keras import layers

### Prepare the data

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()



In [ ]:
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1) vì mạng CNN yêu cầu shape của đầu vào phải là 3 chiều
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

**Nhận xét**:

x_train và x_test sẽ có dạng tensor 4 chiều:

- Chiều thứ nhất: Số lượng mẫu dữ liệu (ví dụ: 60000 cho x_train).
- Chiều thứ hai: Chiều cao của ảnh (28 pixel).
- Chiều thứ ba: Chiều rộng của ảnh (28 pixel).
- Chiều thứ tư: Số kênh màu (1 kênh cho ảnh xám).

Nói cách khác, đầu vào của mô hình là một tập hợp các ma trận (tensor 3 chiều), mỗi ma trận đại diện cho một ảnh.

### Build the model

**Kiến trúc mạng**

Mạng Nơ-ron Tích chập (CNN) được xây dựng bằng cách sử dụng Keras' Sequential API. Nó được thiết kế cho phân loại hình ảnh, đặc biệt là cho tập dữ liệu MNIST (chữ số viết tay).

Dưới đây là phân tích chi tiết về kiến trúc:

1. Lớp đầu vào:

- keras.Input(shape=input_shape): Xác định lớp đầu vào, mong đợi hình ảnh có hình dạng input_shape, là (28, 28, 1) cho hình ảnh thang độ xám MNIST.
2. Các lớp tích chập:

- layers.Conv2D(32, kernel_size=(3, 3), activation="relu"): Lớp tích chập đầu tiên với 32 bộ lọc, mỗi bộ lọc sử dụng kernel 3x3. Nó áp dụng hàm kích hoạt ReLU cho đầu ra.
- layers.Conv2D(64, kernel_size=(3, 3), activation="relu"): Lớp tích chập thứ hai với 64 bộ lọc, cũng sử dụng kernel 3x3 và kích hoạt ReLU.
Các lớp này trích xuất các đặc trưng từ hình ảnh đầu vào bằng cách sử dụng các bộ lọc đã học.

3. Các lớp gộp tối đa:

- layers.MaxPooling2D(pool_size=(2, 2)): Hai lớp gộp tối đa với kích thước vùng gộp là 2x2. Các lớp này giảm mẫu các bản đồ đặc trưng, giảm kích thước không gian và độ phức tạp tính toán của chúng.

4. Lớp làm phẳng:

- layers.Flatten(): Chuyển đổi các bản đồ đặc trưng đa chiều thành một vectơ duy nhất, chuẩn bị dữ liệu cho các lớp được kết nối đầy đủ.

5. Lớp Dropout:

- layers.Dropout(0.5): Áp dụng điều chuẩn Dropout với tỷ lệ 0.5. Điều này sẽ ngẫu nhiên đặt một phần các đơn vị đầu vào thành 0 trong quá trình huấn luyện, giúp ngăn ngừa hiện tượng quá khớp.

6. Lớp đầu ra:

- layers.Dense(num_classes, activation="softmax"): Lớp được kết nối đầy đủ cuối cùng với num_classes (10 cho MNIST) đơn vị đầu ra. Nó sử dụng hàm kích hoạt softmax để tạo ra phân phối xác suất trên các lớp, đại diện cho xác suất dự đoán cho mỗi chữ số.


In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)
#Mặc dù một số neuron bị bỏ qua, hình dạng của đầu vào và đầu ra của 
# lớp Dropout vẫn giữ nguyên. Dropout chỉ đơn giản là làm cho một số 
# phần tử của vector đầu vào bằng 0, nhưng không làm thay đổi kích thước 
# hay số chiều của vector.
model.summary()

### Train the model


In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


### Evaluate the trained model

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])